In [1]:
#!pip install opencv-python
import numpy as np

import cv2
from matplotlib import pyplot as plt
from os import walk

files = []
target = []
data = []
X_treinamento = []
y_treinamento = []
X_teste = []
y_teste = []
qtdImagens = 0
#Adicionando imagens saudaveis no dataset 
#Carregando set e imagens para treinamento
path = 'dataset/train/saudavel/'
for (dirpath, dirnames, filenames) in walk(path):
  for file in filenames:
    img = cv2.imread('dataset/train/saudavel/' + file)
    img = cv2.resize(img, (100,100))
    #print(len(filenames))
    X_treinamento.append(np.asarray(img).reshape(-1))
    y_treinamento.append(0) # 1 quando é doente, 0 quando é saudável
  break
print(len(data))
#Adicionando imagens doentes no dataset 
path = 'dataset/train/doentes/'
for (dirpath, dirnames, filenames) in walk(path):
  qtdImagens = qtdImagens + len(filenames)
  for file in filenames:
    img = cv2.imread('dataset/train/doentes/' + file)
    img = cv2.resize(img, (100,100))
    X_treinamento.append(np.asarray(img).reshape(-1))
    y_treinamento.append(1) # 1 quando é doente, 0 quando é saudável
  break

#--------------------------------------------------------------------------------------
#Carregando set e imagens para teste
path = 'dataset/test/saudavel'
for (dirpath, dirnames, filenames) in walk(path):
  #files.extend(filenames)
  #img = cv2.imread('imgs/saudaveis/' + str(filenames),0)
  #print( str(filenames))
  qtdImagens = len(filenames)
  for file in filenames:
    img = cv2.imread('dataset/test/saudavel/' + file)
    print('dataset/test/saudavel/' + file)
    img = cv2.resize(img,(100,100))
    #print(len(filenames))
    X_teste.append(np.asarray(img).reshape(-1))
    y_teste.append(0) # 1 quando é doente, 0 quando é saudável
  break


#Adicionando imagens doentes no dataset 
path = 'dataset/test/doentes'
for (dirpath, dirnames, filenames) in walk(path):
  qtdImagens = qtdImagens + len(filenames)
  for file in filenames:
    img = cv2.imread('dataset/test/doentes/' + file)
    img = cv2.resize(img,(100,100))
    X_teste.append(np.asarray(img).reshape(-1))
    y_teste.append(1) # 1 quando é doente, 0 quando é saudável
  break


0
dataset/test/saudavel/4273549-SAUDAVEL.jpg
dataset/test/saudavel/4273550-SAUDAVEL.jpg
dataset/test/saudavel/DSC_0003-SAUDAVEL.jpg
dataset/test/saudavel/DSC_0004-SAUDAVEL.jpg
dataset/test/saudavel/DSC_0019-SAUDAVEL.jpg
dataset/test/saudavel/DSC_0020-SAUDAVEL.jpg
dataset/test/saudavel/DSC_0022-SAUDAVEL.jpg


In [2]:
X_treinamento = np.float64(np.array(X_treinamento))

y_treinamento = np.int32(np.array(y_treinamento))


X_teste = np.float64(np.array(X_teste))
y_teste = np.int32(np.array(y_teste))
X_treinamento.shape

(80, 30000)

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#import tensorflow as tf

Instructions for updating:
non-resource variables are not supported in the long term


In [4]:
def cria_rede(features, labels, mode):
    # batch_size, largura, altura, canais
    entrada = tf.reshape(features['X'], [-1, 100, 100, 3])
    
    # recebe [batch_size, 100, 100, 1]
    # retorna [batch_size, 100, 100, 32]
    convolucao1 = tf.layers.conv2d(inputs = entrada, filters = 32, kernel_size=[2,2], activation = tf.nn.relu,
                                  padding = 'same')
    # recebe [batch_size, 100, 100, 32]
    # retorna [batch_size, 50, 50, 32]
    pooling1 = tf.layers.max_pooling2d(inputs = convolucao1, pool_size = [2,2], strides = 2)
    
    # recebe [batch_size, 50, 50, 32]
    # retorna [batch_size, 50, 50, 64]
    convolucao2 = tf.layers.conv2d(inputs = pooling1, filters = 64, kernel_size = [2,2], activation = tf.nn.relu,
                                  padding = 'same')
    # recebe [batch_size, 50, 50, 64]
    # retorna [batch_size, 25, 25, 64]
    pooling2 = tf.layers.max_pooling2d(inputs = convolucao2, pool_size = [2,2], strides = 2)
    
    # recebe [batch_size, 25, 25, 64]
    # retornar [batch_size, 3136]
    flattening = tf.reshape(pooling2, [-1, 25 * 25 * 64])
    
    # 3136 (entradas) -> 1024 (oculta) -> 10 (saída)
    # recebe [batch_size, 3136]
    # retornar [batch_size, 1024]
    densa = tf.layers.dense(inputs = flattening, units = 1024, activation = tf.nn.relu)
    
    # dropout
    dropout = tf.layers.dropout(inputs = densa, rate = 0.2, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # recebe [batch_size, 1024]
    # retornar [batch_size, 10]
    saida = tf.layers.dense(inputs = dropout, units = 2)
    
    # 0.2 0.2 0.6 - 2
    previsoes = tf.argmax(saida, axis = 1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions = previsoes)
    
    erro = tf.losses.sparse_softmax_cross_entropy(labels = labels, logits = saida)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        otimizador = tf.train.AdamOptimizer(learning_rate = 0.0001)
        treinamento = otimizador.minimize(erro, global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, loss = erro, train_op = treinamento)
    
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metrics_ops = {'accuracy': tf.metrics.accuracy(labels = labels, predictions = previsoes)}
        return tf.estimator.EstimatorSpec(mode = mode, loss = erro, eval_metric_ops = eval_metrics_ops)
    

In [5]:
classificador = tf.estimator.Estimator(model_fn = cria_rede)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Lucas\\AppData\\Local\\Temp\\tmpz7prf1us', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
funcao_treinamento = tf.estimator.inputs.numpy_input_fn(x = {'X': X_treinamento}, y = y_treinamento,
                                                       batch_size = 5, num_epochs = None, shuffle = False)
classificador.train(input_fn=funcao_treinamento, steps = 100)



Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.


C:\Users\Lucas\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\legacy_tf_layers\convolutional.py:536: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
C:\Users\Lucas\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
C:\Users\Lucas\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\legacy_tf_layers\pooling.py:554: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  warnings.warn('`tf.layers.max_pooling2d` is deprecated and '
C:\Users\Lucas\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\legacy_tf_lay

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Lucas\AppData\Local\Temp\tmpz7prf1us\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 40.18126678466797, step = 1
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 100...
INFO:tensorflow:Saving checkpoints for 100 into C:\Users\Lucas\AppData\Local\Temp\tmpz7prf1us\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 100...
INFO:tensorflow:Loss for final step: 0.0.


In [7]:
funcao_teste = tf.estimator.inputs.numpy_input_fn(x = {'X': X_teste}, y = y_teste, num_epochs = 1,
                                                      shuffle = False)
resultados = classificador.evaluate(input_fn=funcao_teste)
resultados

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-11-03T15:23:55
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Lucas\AppData\Local\Temp\tmpz7prf1us\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.07343s
INFO:tensorflow:Finished evaluation at 2021-11-03-15:23:56
INFO:tensorflow:Saving dict for global step 100: accuracy = 0.84444445, global_step = 100, loss = 732.68756
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 100: C:\Users\Lucas\AppData\Local\Temp\tmpz7prf1us\model.ckpt-100


{'accuracy': 0.84444445, 'loss': 732.68756, 'global_step': 100}

In [8]:
X_imagem_teste = X_teste[10]
X_imagem_teste.shape

(30000,)

In [9]:
X_imagem_teste = X_imagem_teste.reshape(1,-1)
X_imagem_teste.shape

(1, 30000)

In [10]:
funcao_previsao = tf.estimator.inputs.numpy_input_fn(x = {'X': X_imagem_teste}, shuffle = False)
pred = list(classificador.predict(input_fn = funcao_previsao))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Lucas\AppData\Local\Temp\tmpz7prf1us\model.ckpt-100
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [11]:

print(y_teste[10].reshape(1,-1))
pred

[[1]]


[1]